In [45]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from utils.embedding_utils import *
from classifiers.CustomClassifiers import AggregateClassifier,AggregateClassifierGrid
from itertools import combinations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
embedding_dir=os.path.join(os.getcwd(),'outputs',"stats","main_54")
full_stat_df=pd.read_csv(os.path.join(embedding_dir,"embedding_stats_avg5.csv"))
output_dir=os.path.join(embedding_dir,"stat_analysis")

In [78]:
def run_embedding_classiciation(stat_df,X_weight,use_prob=True,min_combos=3,
                                max_combos=4,skip_w=True,pred_col='diagnosis',conditionals=[]):
    graph_labels = stat_df[pred_col].values
    data_names=['W','rad','btw','coh']
#     data_names=['rad','btw','coh']
    features=[]
    for n in data_names:
        if n in ['rad','btw','coh']:
            cols_to_use=[c for c in stat_df.columns if n in c.lower()]
            if use_prob:
                cols_to_use=[c for c in cols_to_use if 'prob' in c.lower()]
            else:
                cols_to_use=[c for c in cols_to_use if 'prob' not in c.lower()]
            n_df=np.array(stat_df[cols_to_use])
            data,y_comb,pat_to_indx=combine_patient_feats(n_df,graph_labels,stat_df,conditional=conditionals)
            print(n_df,'NF')
        else:
            n_df=X_weight
            data,y_comb,pat_to_indx=combine_patient_feats(n_df,graph_labels,stat_df,conditional=conditionals)
            Wa=data
            
            
#         data,y_comb,pat_to_indx=combine_patient_feats(n_df,graph_labels,stat_df,conditional=conditionals)
        features.append(data)
        
    
        

#     Wa=X_weight_comb
#     names=['W','X_rc','X_bc','X_wc']
    indices=np.array([i for i in range(len(data_names))])
    feature_combinations=[]
    name_combinations=[]
    list_combinations = list()

    for n in range(min_combos,max_combos+1):
        for combo in combinations(indices, n):
            print(combo)
            feat_list=[features[i] for i in combo]
            name_list=[data_names[i] for i in combo]
            print(name_list)
            feat_combines=np.hstack(feat_list)
            print(feat_combines.shape)
            feature_combinations.append(feat_combines)
            name_combinations.append(name_list)

    if not skip_w:
        feature_combinations=[Wa]+feature_combinations
        name_combinations=[['W']]+name_combinations
        
    return feature_combinations,name_combinations,y_comb
    

In [79]:
conditionals = ((full_stat_df['CogTr']==1))
fit_method='concatenate'
predict_method='same'
# run_combos_list(model_type=LogisticRegression,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb)  


In [80]:
import argparse
parser = argparse.ArgumentParser(description='RiemannianGNN')
args, _ = parser.parse_known_args()
MEGLpParams.add_params(parser)
args = parser.parse_args(args=[])

In [81]:
BAND_TO_INDEX = {'theta':0,'alpha':1,'beta1':2,'beta2':3,'beta':4,'gamma':5}
METRIC_TO_INDEX = {'plv':0,'ciplv':1}


band_adj = BAND_TO_INDEX['alpha']
metric_adj = METRIC_TO_INDEX[args.metric]
raw_scans = np.load(args.raw_scan_file)
scan_data = raw_scans[:,band_adj,metric_adj] ### this is not changed by any hyperparamters

graph_ids = full_stat_df['Scan Index'].values
scan_data = raw_scans[:,band_adj,metric_adj] ### this is not changed by any hyperparamters
train_scans = scan_data[graph_ids.astype(int) ]
X_weight = np.array([a[np.triu_indices(a.shape[0])] for a in train_scans])

In [82]:

feature_combinations,name_combinations,y_comb=run_embedding_classiciation(full_stat_df,X_weight,skip_w=False,
                                                                   use_prob=False,min_combos=2,max_combos=4,conditionals=conditionals)

conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: CogTr, Length: 180, dtype: bool
NO SHUFFLE
conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    False
179    False
Name: CogTr, Length: 180, dtype: bool
NO SHUFFLE
[[1.14856664 2.0953061  2.18031072 ... 2.05160935 1.42537626 2.26457091]
 [1.12309426 2.06000353 2.21085388 ... 2.09441905 1.45372466 2.33475558]
 [1.21732124 1.98617806 2.04157465 ... 1.89624364 1.33290461 1.99036129]
 ...
 [1.19672428 2.10935528 2.21695379 ... 2.04229307 1.52801455 2.00044454]
 [1.30283475 1.89239307 1.91787272 ... 1.98593059 1.42585129 1.99628657]
 [1.29968796 1.78858962 1.84196672 ... 1.9000061  1.39210995 1.90546139]] NF
conditational
0       True
1       True
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178    Fal

In [77]:
# feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[0],X_weight,
#                                                                    use_prob=True,min_combos=3,max_combos=4)

In [83]:
for f in feature_combinations:
    print(f.shape)

(46, 4095, 2)
(46, 4103, 2)
(46, 4123, 2)
(46, 4103, 2)
(46, 36, 2)
(46, 16, 2)
(46, 36, 2)
(46, 4131, 2)
(46, 4111, 2)
(46, 4131, 2)
(46, 44, 2)
(46, 4139, 2)


In [86]:
fit_method='split_sample'
predict_method='prediction_mean'
run_combos_list(model_type=SVC,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,
                agg_pred=predict_method,y_comb=y_comb,use_nested=False)  
# run_combos_list(model_type=SVC,names=name_combinations,
#                 feature_list=feature_combinations,times=1,
#                 use_agg=True,agg_fit=fit_method,
#                 agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

(46,) y?
(46, 4095, 2) X
None PASS
0.681510101010101 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': 'rbf'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 4095, 2) new
(46,) clone
(46, 4095, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.68      0.77      0.72        22
           2       0.76      0.67      0.71        24

    accuracy                           0.72        46
   macro avg       0.72      0.72      0.72        46
weighted avg       0.72      0.72      0.72        46

0.717391304347826 accuracy
0.7159090909090908 roc
(0.7172576832151301, 0.7159090909090908) res
(46,) y?
(46, 4103, 2) X
None PASS
0.7235083250083251 50 num_split?
{'C': 1, 'class_weight': 'balanced', 'gamma': 0.5, 'kernel': '

              precision    recall  f1-score   support

           1       0.83      0.68      0.75        22
           2       0.75      0.88      0.81        24

    accuracy                           0.78        46
   macro avg       0.79      0.78      0.78        46
weighted avg       0.79      0.78      0.78        46

0.782608695652174 accuracy
0.7481060606060606 roc
(0.7788461538461537, 0.7481060606060606) res
(46,) y?
(46, 44, 2) X
None PASS
0.6740173715173715 50 num_split?
{'C': 0.3, 'class_weight': 'balanced', 'degree': 2, 'kernel': 'poly'} BEST PARAMS??

NEW!
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] EXAMPLE Y
(46,) EXAMPLE Y
(46, 44, 2) new
(46,) clone
(46, 44, 2)  X shape???sss
LeaveOneOut()
[1 2 2 2 1 1 1 1 2 1 2 1 2 1 1 1 1 1 1 2 1 1 2 2 1 2 2 2 1 1 2 2 2 2 2 2 1
 2 2 2 2 1 2 2 1 1] YYY
              precision    recall  f1-score   support

           1       0.67      0.64      0.65        22
           2       0.68 

In [ ]:
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[1],X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)


In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=2,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=True)  

In [ ]:
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[2],X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=2,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=True)  

In [ ]:
skks
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(all_stat_dicts[2],X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

In [ ]:
feature_combinations,name_combinations,y_comb=run_embedding_classiciation(full_dict,X_weight,
                                                                   use_prob=True,min_combos=3,max_combos=4)

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=3,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb,use_nested=False)  

In [ ]:
num=1
names_short=name_combinations[:num]
feats_short=feature_combinations[:num]

In [ ]:
run_combos_list(model_type=LogisticRegression,names=name_combinations,
                feature_list=feature_combinations,times=1,
                use_agg=True,agg_fit=fit_method,agg_pred=predict_method,y_comb=y_comb)  